In [1]:
import time
import numpy as np

np.random.seed(0)

from lib.linear_algebra import Matrix
from lib.nn import Linear, ReLU, NN
from lib.processing import ColumnNormalizer

In [2]:
# Boston House Price Dataset
# The Boston House Price Dataset involves the prediction of a house price in thousands of dollars given details of the house and its neighborhood.
# It is a regression problem. There are 506 observations with 13 input variables and 1 output variable. The variable names are as follows:

data = []
with open("data/boston_house_prices.data", "rt") as f:
    for line in f.readlines():
        data.append([float(v) for v in line.split()])
data = np.array(data)

In [3]:
np.random.shuffle(data)
split = int(len(data) * 0.8)
train_set = data[:split]
test_set = data[split:]

X_train, y_train = train_set[:, :-1], train_set[:, -1:]
X_test, y_test = test_set[:, :-1], test_set[:, -1:]
X_train = Matrix(X_train)
y_train = Matrix(y_train)
X_test = Matrix(X_test)
y_test = Matrix(y_test)
X_train.dims(), y_train.dims(), X_test.dims(), y_test.dims()

((404, 13), (404, 1), (102, 13), (102, 1))

In [4]:
normalizer = ColumnNormalizer()
normalizer.fit(X_train)
X_train = normalizer.transform(X_train)
X_test = normalizer.transform(X_test)
X_train.dims(), X_test.dims()

((404, 13), (102, 13))

In [5]:
nn = NN([
    Linear(13, 4),
    ReLU(),
    Linear(4, 1),
])

In [6]:
time_point = time.time()

for i in range(2001):
    out = nn(X_train)
    L = sum([i[0] for i in (y_train - out) ** 2]) / y_train.dims()[0]
    if i % 100 == 0:
        elapsed_time = int(time.time() - time_point)
        time_point = time.time()
        print(f"{i} | {L.data:.2f} | {elapsed_time}s")    

    for p in nn.params():
        p.zero_grad()
    L.grad = 1
    L.backward()

    for p in nn.params():
        for v in p.all_values():
            v.data -= 0.001 * v.grad

0 | 626.32 | 1s
100 | 47.83 | 236s
200 | 32.07 | 230s
300 | 26.48 | 240s
400 | 23.86 | 253s
500 | 22.20 | 238s
600 | 20.86 | 224s
700 | 19.75 | 226s
800 | 18.88 | 252s
900 | 18.17 | 220s
1000 | 17.59 | 221s
1100 | 17.11 | 221s
1200 | 16.69 | 256s
1300 | 16.34 | 250s
1400 | 16.04 | 248s
1500 | 15.77 | 237s
1600 | 15.57 | 241s
1700 | 15.40 | 231s
1800 | 15.25 | 225s
1900 | 15.10 | 225s
2000 | 14.91 | 248s


In [7]:
out = nn(X_test)
L = sum([i[0] for i in (y_test - out) ** 2]) / y_test.dims()[0]
L

{c8c89081, 12.1, 0}

In [8]:
[(round(float(v1),1), float(v2)) for v1, v2 in zip([v[0].data for v in out.values], [v[0].data for v in y_test])][:10]

[(23.5, 20.7),
 (41.8, 39.8),
 (17.5, 17.8),
 (20.1, 19.6),
 (11.5, 14.9),
 (24.1, 22.0),
 (47.4, 48.8),
 (23.3, 25.0),
 (44.1, 48.5),
 (27.4, 23.9)]